In [1]:
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as FF
import plotly
#plotly.tools.set_credentials_file(username='ninjamouse', api_key='LejqbK7NWhPU7zXoMuhw')

import numpy as np
import pandas as pd

df = pd.read_csv('first_attack.csv')

In [2]:
df.Source.value_counts()[0:5]

192.168.4.120      15032
74.63.40.21         5735
192.168.56.1        4733
75.127.97.72        4218
174.133.126.130     3747
Name: Source, dtype: int64

In [3]:
df.Destination.value_counts()[0:5]

192.168.4.120    23440
75.127.97.72      4719
192.168.56.1      4218
74.63.40.21       3170
192.168.2.110     2767
Name: Destination, dtype: int64

In [4]:
dest=df[df.Destination=='75.127.97.72']
dest.head()

,No.,Time,Source,Destination,Protocol,Length,Info
31173,31174,"2010-06-12 06:54:06,347684",192.168.56.1,75.127.97.72,TCP,74,52998 > 80 [SYN] Seq=0 Win=29200 Len=0 MSS=1...
31175,31176,"2010-06-12 06:54:06,348033",192.168.56.1,75.127.97.72,TCP,66,52998 > 80 [ACK] Seq=1 Ack=1 Win=29696 Len=0...
31176,31177,"2010-06-12 06:54:06,348044",192.168.56.1,75.127.97.72,HTTP,710,GET /drupal7/ HTTP/1.1
31184,31185,"2010-06-12 06:54:06,552973",192.168.56.1,75.127.97.72,TCP,66,52998 > 80 [ACK] Seq=645 Ack=4345 Win=37888 ...
31186,31187,"2010-06-12 06:54:06,553056",192.168.56.1,75.127.97.72,TCP,66,52998 > 80 [ACK] Seq=645 Ack=4444 Win=37888 ...


In [5]:
protocol=dest[dest.Protocol=="HTTP"]
protocol.head()

,No.,Time,Source,Destination,Protocol,Length,Info
31176,31177,"2010-06-12 06:54:06,348044",192.168.56.1,75.127.97.72,HTTP,710,GET /drupal7/ HTTP/1.1
31192,31193,"2010-06-12 06:54:06,585868",192.168.56.1,75.127.97.72,HTTP,749,GET /drupal7/sites/default/files/styles/medium...
31203,31204,"2010-06-12 06:54:06,586953",192.168.56.1,75.127.97.72,HTTP,734,GET /drupal7/sites/default/files/styles/medium...
31206,31207,"2010-06-12 06:54:06,587263",192.168.56.1,75.127.97.72,HTTP,733,GET /drupal7/sites/default/files/styles/medium...
31208,31209,"2010-06-12 06:54:06,587412",192.168.56.1,75.127.97.72,HTTP,684,GET /drupal7/misc/feed.png HTTP/1.1


In [6]:
dest.Protocol.value_counts()
dest.Info[0:5]

31173    52998  >  80 [SYN] Seq=0 Win=29200 Len=0 MSS=1...
31175    52998  >  80 [ACK] Seq=1 Ack=1 Win=29696 Len=0...
31176                              GET /drupal7/ HTTP/1.1 
31184    52998  >  80 [ACK] Seq=645 Ack=4345 Win=37888 ...
31186    52998  >  80 [ACK] Seq=645 Ack=4444 Win=37888 ...
Name: Info, dtype: object

In [8]:
trace1=go.Scatter(x=dest['Time'],y=dest['Length'],mode='markers')
d = dest.query('(Length>300)&(Length<600)')
trace2=go.Scatter(x=d['Time'],y=d['Length'], mode='lines')
data=[trace1, trace2]
py.iplot(data)

In [9]:
from sklearn import linear_model

In [ ]:
X = df

# Treat the problem as a pure classification problem.

X['Protocol'] = X['Protocol'].astype('category')
# Could use float, but it doesn't make much sense
#X['Source'] = X['Source'].apply(lambda x: float(x.replace('.','')))

# Convert IPs to classes
X['Destination'] = X['Destination'].astype('category')
X['Source'] = X['Source'].astype('category')

# Convert info to classes as well
X['Info'] = X['Info'].astype('category')  # ToDo split info in relevant columns

# Drop the time series for now, just to play with a classifier
X = X.drop(['Time'], axis=1)

In [ ]:
# Set some entries as 1 'attack' and 0 no attack
# The ones you defined in d above, are entries where an attack was plausible. Everything else is clean
Y = [0 if i not in d.index.values else 1 for i in df.index.values]

In [ ]:
catColumns = X.select_dtypes(['category']).columns
X[catColumns] = X[catColumns].apply(lambda x: x.cat.codes)

In [ ]:
clf = linear_model.SGDClassifier(max_iter=1000, tol=1e-3)
clf.fit(X, Y)

In [ ]:
# ## TODO

# df = pd.read_csv('second_attack.csv')

# X_test = df

# ## apply transformation methods as above

# pred = clf.predict(X_test)

In [ ]:
## TODO
## Compute 'd' the same way you computed it in the first case
## Plot d + pred
## Compare overalp between d and pred

In [10]:
# 1. Pot sa filtrez dupa 2 campuri? 
# Vreau spre exemplu sa vad cele mai multe conexiuni intre 2 IP-uri, 
# dar as vrea sa vad un top, o sortare facuta initial. 
# Nu cunosc care sunt IP-urile care au discutat cel mai mult. 
# Pot sa sortez dupa IP-ul destinatie sau sursa, 
# dar as vrea o sortare facuta dupa numarul de conexiuni intre 2 IP-uri.

df['Destination'] = df['Destination'].astype('str')
df['Source'] = df['Source'].astype('str')

# # Dureaza un pic pentru tot datasetul.
sourceDict = { sourceIp: df[df.Source == sourceIp]['No.'].count() for sourceIp in df.Source[0:10]}

# Poti mai eficient
df.drop(['Time', 'Protocol', 'Length', 'Info'], axis=1)\
            .groupby(["Source", 'Destination']) \
            .count() \
            .sort_values(['No.'], ascending=False)\
            .head()

,,No.
Source,Destination,
74.63.40.21,192.168.4.120,5735
192.168.56.1,75.127.97.72,4719
75.127.97.72,192.168.56.1,4218
174.133.126.130,192.168.4.120,3747
192.168.4.120,74.63.40.21,3170


In [11]:
# 2. In coloana de info sunt mai multe detalii. 
# As putea cumva sa accesez continutul acelei coloane? 
# Spre exemplu vreau sa vad doar requesturile (GET) si 
# nu si (ACK sau RST etc). Sa pot parsa, sau asta ar 
# trebui facut inainte de a salva captura ca .csv?

df['Info'] = df['Info'].astype('str')

# Poti sa faci ce operatie vrei tu pe fiecare rand, dupa ce faci cast to string la coloana
df[df['Info'].str.contains('GET')].head()

,No.,Time,Source,Destination,Protocol,Length,Info
4,5,"2010-06-12 06:50:00,064234",192.168.4.120,208.122.28.10,HTTP,140,GET /profile_images/159516/rollinsliar_small.j...
6,7,"2010-06-12 06:50:00,066804",192.168.4.120,88.208.248.162,HTTP,245,GET /Car-Products/Bike-Racks/Bike-Racks-Thule-...
10,11,"2010-06-12 06:50:00,092368",192.168.2.110,67.220.214.50,HTTP,112,GET /images/rss2.gif HTTP/1.1
12,13,"2010-06-12 06:50:00,121749",192.168.4.120,208.122.28.2,HTTP,149,GET /profile_images/228715/Logo_Baseball_Canad...
18,19,"2010-06-12 06:50:00,180490",192.168.4.120,208.122.28.4,HTTP,143,GET /profile_images/259115/discrimination_smal...
